### Example of comparing noise and signal Fourier amplitude spectra for [M 5.4 - 31 km SSE of Karyes, Greece Earthquake](https://earthquake.usgs.gov/earthquakes/eventpage/us6000c1rq/executive)
2020-09-26 22:50:25 (UTC)<br><br>
Downloading 10 min of noise and 10 min of signal of vertical component recording (**HHZ** component) by station **OGCN** (network code: **FR**) from the **RESIF** data center<br>

Computing Fourier transforms and plotting spectral amplitudes 

---



### Step 1: Reading and plotting original seismogram

In [ ]:
#------------------------ importing basic packages
import matplotlib.pyplot as plt
import numpy as np
#------------------------ importing ObsPy functions
from obspy import read
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

#------------------ plotting mode
%matplotlib widget
#----------------------------


#------------------------ selecting an FDSN datacenter
client = Client('RESIF')

#--------------- downloading noise window
tstart = UTCDateTime("2020-09-26T22:43:10.000")
t_duration = 30*60
st = client.get_waveforms("G", "SSB", "*", "HHZ", tstart, tstart + t_duration, attach_response=True)
signal = st[0]

signal.detrend('linear')

dts = signal.stats.delta
npts = signal.stats.npts
times = dtn*(np.linspace(1,npts,npts)-1+nptn)

plt.figure()
plt.plot(times,signal.data,'k')
plt.title(signal.stats.station)
plt.xlabel('time (s)')
plt.ylabel('counts')
plt.show()

### Step 2: Comparing "noise" and "signal" wndows

In [ ]:
#------------------------ importing basic packages
import matplotlib.pyplot as plt
import numpy as np
#------------------------ importing ObsPy functions
from obspy import read
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

#------------------ plotting mode
%matplotlib widget
#----------------------------


#------------------------ selecting an FDSN datacenter
client = Client('RESIF')


#-------------------- defining duration of the downloaded time series in sec
t_duration =11*60


#--------------- downloading noise window
tstart = UTCDateTime("2020-09-26T22:43:10.000")
st1 = client.get_waveforms("G", "SSB", "*", "HHZ", tstart, tstart + t_duration, attach_response=True)
noise = st1[0]

#--------------- downloading signal window
tstart = UTCDateTime("2020-09-26T22:54:10.000")
st1 = client.get_waveforms("G", "SSB", "*", "HHZ", tstart, tstart + t_duration, attach_response=True)
signal = st1[0]

dtn = noise.stats.delta
nptn = noise.stats.npts
timen = dtn*(np.linspace(1,nptn,nptn)-1)

dts = signal.stats.delta
npts = signal.stats.npts
times = dtn*(np.linspace(1,npts,npts)-1+nptn)

plt.figure()
plt.plot(timen,noise.data,'b')
plt.plot(times,signal.data,'r')
plt.title(signal.stats.station)
plt.xlabel('time (s)')
plt.ylabel('counts')
plt.show()


noise.detrend()
signal.detrend()



#-----------------------------------------------
# function to compute Fourier spectra
#-----------------------------------------------
def signal_fft1d(sig,dt):
    npt = np.size(sig)
    spe = np.fft.fft(sig)
    freq = np.fft.fftfreq(npt,dt)
    sp_amp = np.sqrt(spe.real**2+spe.imag**2)
    sp_pha = np.arctan2(spe.imag, spe.real)
    npt_spe = int(npt/2)
    return npt_spe, sp_amp[0:npt_spe],sp_pha[0:npt_spe],freq[0:npt_spe]

#-----------------------------------------------
# function to smooth an array
#-----------------------------------------------
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

nspe, n_spamp, sppha, fr = signal_fft1d(noise.data,noise.stats.delta)
nspe, s_spamp, sppha, fr = signal_fft1d(signal.data,signal.stats.delta)


plt.figure()
plt.loglog(fr,smooth(n_spamp,20),'b')
plt.loglog(fr,smooth(s_spamp,20),'r')
plt.xlim(.005,20)
plt.xlabel('frequency(Hz)')
plt.show()



### Step 3: Filtering and plotting

In [ ]:
#------------------------ importing basic packages
import matplotlib.pyplot as plt
import numpy as np
#------------------------ importing ObsPy functions
from obspy import read
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

#------------------ plotting mode
%matplotlib widget
#----------------------------

signal = st[0].copy()

signal.detrend('linear')


signal.filter("bandpass", freqmin=0.03, freqmax=.15, corners=4, zerophase=True)
#signal.filter("bandpass", freqmin=.6, freqmax=3, corners=4, zerophase=True)


dts = signal.stats.delta
npts = signal.stats.npts
times = dtn*(np.linspace(1,npts,npts)-1+nptn)

plt.figure()
plt.plot(times,signal.data,'k')
plt.title(signal.stats.station)
plt.xlabel('time (s)')
plt.ylabel('counts')
plt.show()